<a href="https://colab.research.google.com/github/rodiwaa/learnings-pocs/blob/main/notebooks/adk_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# plan
# - create simple greetin_agent
# - create tool to print hello or hey
# - wrap in functool
# - attach to greeting_agent
# - print
# - stream

# summary
"""
query - create travel itenerary for <PLACE>
agent workflow
  - parallel/ROOT/core agent (naming needs to be fixed in final system)
    - city weather
    - news, including city local news
  ---> merger/LLMAgent - merges outputs of news, weather from parallel agent into singular report

running agents?
  - done run indi agents.
  - go top-down --> run root/core/orchestrator only
  - all sub-agents run via flow
"""

'\nquery - create travel itenerary for <PLACE>\nagent workflow\n  - parallel/ROOT/core agent (naming needs to be fixed in final system)\n    - city weather\n    - news, including city local news\n  ---> merger/LLMAgent - merges outputs of news, weather from parallel agent into singular report \n\nrunning agents?\n  - done run indi agents.\n  - go top-down --> run root/core/orchestrator only\n  - all sub-agents run via flow\n'

In [9]:
! pip install google-adk google-generativeai

In [10]:
from google.adk.agents import Agent, ParallelAgent, LlmAgent, SequentialAgent
from google.adk.tools import google_search
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService, Session
from google.genai.types import Content, Part
from IPython.display import display, Markdown

In [11]:
from getpass import getpass
# from google.generativeai import genai
import google.generativeai as genai
import os

api_key = getpass("enter google api_key: ")

genai.configure(api_key=api_key)

os.environ["GOOGLE_API_KEY"] = api_key

print("google api key set")

enter google api_key: ··········
google api key set


In [5]:
# agent

# session

# userid

# runner

# run

In [14]:
# --- Agent Definition ---
# use for defining wrather, news agents only
# agents
def create_weather_agent():
    """Create the city weather forecast agent"""
    return Agent(
        name="weather_agent",
        model="gemini-2.5-flash",
        description="Agent specialized in fetching day forecast of given place.",
        instruction="""
        You are the "Weather Forecaster" Agent 🚗 - a specialized AI assistant that creates forecast weather.

        Your Mission:
        Get weather forecast for given location. Present in friendly manner.

        Guidelines:
        1. **General Weather**: Focus on general weather conditions like hot, sunny, rainy.
        2. **Temperature**: Create morning, afternoon, and evening forecast for day.
        3. **AQI Pollution Info**: Find relevant info on pollution levels. Mention AQI levels with categories (Good, Safe, Warning, Critical).
        4. **Mood Matching**: Align suggestions with the requested mood (adventurous, relaxing, artsy, etc.).

        RETURN summary in MARKDOWN FORMAT with clear time blocks.
        """,
        tools=[google_search],
        output_key="weather-results" # saves output in state[weather-results], so other components/ agents can access weather data from here. # think tool-context and app-state
    )

def create_news_agent():
    """Specialised agent in finding local news headlines for major categories."""
    return Agent(
        name="news_agent",
        model="gemini-2.5-flash",
        description="Agent specialized in finding news headlines for major categories.",
        instruction="""
        You are the "News" Generator 🚗 - a specialized AI assistant that brings important news for the day.

        Your Mission:
        Get top 2 headlines and new article URL for categories - World, Sports (Cricket), Business, Technology, Local (specified city) only.

        Guidelines:
        1. **Headlines and URL**: Make sure headlines summarise gist of news item. Provide clickable URL.
        2. **Categories**: Stick to given categories only - World, Sports (Cricket), Business, Technology, Local (specified city) only.
        3. **Avoid**: Avoid these topics - Politics, Bollywood.

        RETURN news in MARKDOWN FORMAT with clear healines and working URLs.
        """,
        tools=[google_search],
        output_key="news-results"
    )

weather_agent = create_weather_agent()
news_agent = create_news_agent()
print(f"🧞 Agent '{weather_agent.name}' is created and ready to bring you weather updates!")
print(f"🧞 Agent '{news_agent.name}' is created and ready to bring you news!")

## runs weather, news in parallel, then sequntial to summary/ merge agent
## rename to "core-agent"? let "root-agent" be the seq agent?
# defined below; this is reddundant
# def create_root_agent():
#   """Root agent to run weather, news agents in parallel"""
#   return ParallelAgent(
#       name="root",
#       # model="gemini-2.5-flash",
#       description="Runs weather and news agents in parallel.",
#       sub_agents=[
#           weather_agent, news_agent # yep
#       ]
#   )

# root_agent = create_root_agent() # TODO: FIXME: ambiguous naming conventions. wtf root = orchestrator now!
# print(f"🧞 Agent '{root_agent.name}' is created and ready!")

## incorrect agent creation pattern used; keep siimilar patterns only for future proof "'def create_merger_agent' suggested"
def create_merger_agent():
  return LlmAgent(
     name="WethaerNewsMergerAgent",
     model="gemini-2.5-flash",
     instruction="""You are an AI Assistant responsible for combining news, weather findings into a structured report.

 Your primary task is to synthesize the following results, clearly attributing findings to their source areas. Structure your response using headings for each topic. Ensure the report is coherent and integrates the key points smoothly.

 **Crucially: Your entire response MUST be grounded *exclusively* on the information provided in the 'Input Summaries' below. Do NOT add any external knowledge, facts, or details not present in these specific results.**

 **Input Summaries:**

 *   **News:**
     {news-results}

 *   **Weather:**
     {weather-results}

 **Output Format:**

 ## Summary of News Weather for <CITY_NAME>

 ### News Results
 (Based on news_agent's findings)
 [Synthesize and elaborate *only* on the news summary provided above.]

 ### Weather Results
 (Based on weather_agent's findings)
 [Synthesize and elaborate *only* on the weather summary provided above.]

 ### Overall Summary
 [Provide a brief (1-2 sentence) concluding statement that connects *only* the results presented above.]

 MARKDOWN output *only* the structured report following this format. Do not include introductory or concluding phrases outside this structure, and strictly adhere to using only the provided input summary content.
 """,
     description="Combines findings from news, weather agents into a structured strictly grounded on provided inputs.",
     # No tools needed for merging
     # No output_key needed here, as its direct response is the final output of the sequence
 )

merger_agent = create_merger_agent()
print(f"🧞 Agent '{merger_agent.name}' is created and ready to merge news weather into single report!")



🧞 Agent 'weather_agent' is created and ready to bring you weather updates!
🧞 Agent 'news_agent' is created and ready to bring you news!
🧞 Agent 'WethaerNewsMergerAgent' is created and ready to merge news weather into single report!


In [15]:
## session service creation
session_service = InMemorySessionService()
print("initialised session service")
## consistent user ID across app;
## TODO - better way before prod?
my_user_id = "rodi_adventurer_001"
print("setup dummy user id")

initialised session service
setup dummy user id


In [ ]:
# run both

In [16]:
# helper util to run ANY defined agent
async def run_agent_query(agent: Agent, query: str, session: Session, user_id: str, is_router: bool = False):
    """Initializes a runner and executes a query for a given agent and session."""
    print(f"\n🚀 Running query for agent: '{agent.name}' in session: '{session.id}'...")

    runner = Runner(
        agent=agent, # A-S-A-N
        session_service=session_service,
        app_name=agent.name
    )
    final_response = ""
    try:
       async for event in runner.run_async( # U-S-M-C
            user_id=user_id,
            session_id=session.id,
            new_message=Content(parts=[Part(text=query)], role="user")
        ):

         if event.is_final_response():
                final_response = event.content.parts[0].text
    except Exception as e:
      final_response = f"An error occurred: {e}"

    print("\n" + "-"*50)
    print("✅ Final Response:")
    display(Markdown(final_response))
    print("-"*50 + "\n")

In [17]:
### NOT RUNNING THESE AGENTS INDIVIDUALLY NOW. RUN ROOT AGENT INSTEAD ###
### RUN FOR TESTING ONLY
async def run_weather_agent():
    # Create a new, single-use session for this query
    weather_session = await session_service.create_session(
        app_name=weather_agent.name,
        user_id=my_user_id
    )

    # Note the new budget constraint in the query!
    query = "Find me next 3 days weather forecast for Mumbai"
    print(f"🗣️ User Query: '{query}'")

    await run_agent_query(weather_agent, query, weather_session, my_user_id)

await run_weather_agent()

🗣️ User Query: 'Find me next 3 days weather forecast for Mumbai'

🚀 Running query for agent: 'weather_agent' in session: '33925916-d5ae-457a-89ea-064be01ff882'...

--------------------------------------------------
✅ Final Response:


Here's your friendly 3-day weather forecast for Mumbai! It looks like you'll be experiencing warm, dry conditions with some air quality concerns to keep in mind.

### 🗓️ Saturday, January 31, 2026

**General Weather:** Mumbai is expected to have a hazy and cloudy day with dense smog affecting visibility, especially in the morning. The day will remain dry.

**Temperature:**
*   **Morning (6 AM - 12 PM):** Starting cool and hazy, temperatures will be around 24°C to 25°C.
*   **Afternoon (12 PM - 6 PM):** Expect warmer conditions with temperatures around 25°C. The high for the day is expected to be around 27.4°C to 31°C.
*   **Evening (6 PM - 12 AM):** Temperatures will be around 24°C to 25°C.

**AQI Pollution Info:** The Air Quality Index (AQI) for Mumbai on Saturday is categorized as **Unhealthy** with an AQI of 218. PM2.5 and PM10 levels are elevated, and individuals with respiratory issues should reduce outdoor exposure. Over 85% of the city reported AQI levels above 300, making the air unsafe even for healthy individuals, so limiting outdoor activity is advisable.

### 🗓️ Sunday, February 1, 2026

**General Weather:** Get ready for a sunny and clear day!

**Temperature:**
*   **Morning (6 AM - 12 PM):** The day will start around 24°C to 25°C.
*   **Afternoon (12 PM - 6 PM):** Temperatures will climb to around 26°C, with a daily high expected around 28°C to 28.3°C.
*   **Evening (6 PM - 12 AM):** Expect pleasant conditions with temperatures around 25°C.

**AQI Pollution Info:** The AQI for Sunday is forecast to be **Unhealthy** with a value of 142. Sensitive groups may immediately feel health effects, and healthy individuals might experience difficulty breathing or throat irritation with prolonged exposure. It's best to limit outdoor activities.

### 🗓️ Monday, February 2, 2026

**General Weather:** Another lovely day with sunny and clear skies.

**Temperature:**
*   **Morning (6 AM - 12 PM):** Temperatures will be around 24°C to 25°C.
*   **Afternoon (12 PM - 6 PM):** The afternoon will see temperatures around 25°C, with a daily high expected around 28°C.
*   **Evening (6 PM - 12 AM):** Temperatures will be around 25°C to 26°C.

**AQI Pollution Info:** The AQI for Monday is expected to be **Unhealthy** with a value of 139. Similar to Sunday, sensitive groups should be cautious, and healthy individuals may experience discomfort during extended outdoor activities. Limiting your time outdoors is recommended.

--------------------------------------------------



In [19]:
### NOT RUNNING THESE AGENTS INDIVIDUALLY NOW. RUN ROOT AGENT INSTEAD ###
### RUN FOR TESTING ONLY
async def run_news_agent():
    # Create a new, single-use session for this query
    news_session = await session_service.create_session(
        app_name=news_agent.name,
        user_id=my_user_id
    )

    # Note the new budget constraint in the query!
    query = "Find me top news headlines for the day for Mumbai"
    print(f"🗣️ User Query: '{query}'")

    await run_agent_query(news_agent, query, news_session, my_user_id)

await run_news_agent()

🗣️ User Query: 'Find me top news headlines for the day for Mumbai'

🚀 Running query for agent: 'news_agent' in session: '15083ff3-b023-4e3c-ae1d-d04fec99dd45'...

--------------------------------------------------
✅ Final Response:


Here are the top news headlines for today, January 31, 2026, for the specified categories:

### World
1.  **Trump Warns UK on China Dealings**
    [https://bestcolleges.com/news/school-assembly-news-headlines-today-january-31/](https://bestcolleges.com/news/school-assembly-news-headlines-today-january-31/)
2.  **Iran Army Chief Issues Warning to US and Israel Amid Heightened Tensions**
    [https://www.thehindu.com/news/national/top-news-of-the-day-sunetra-pawar-sworn-in-as-first-woman-deputy-cm-of-maharashtra-rybakina-becomes-first-kazakh-to-win-australian-open-womens-singles-title-and-more/article69234812.ece](https://www.thehindu.com/news/national/top-news-of-the-day-sunetra-pawar-sworn-in-as-first-woman-deputy-cm-of-maharashtra-rybakina-becomes-first-kazakh-to-win-australian-open-womens-singles-title-and-more/article69234812.ece)

### Sports (Cricket)
1.  **India Seals 4-1 T20I Series Victory Against New Zealand with Ishan Kishan Ton and Arshdeep Singh Fifer**
    [https://sports.ndtv.com/cricket/india-vs-new-zealand-5th-t20i-highlights-ishan-kishan-ton-arshdeep-singh-fifer-seal-4-1-series-win-for-india-7649774](https://sports.ndtv.com/cricket/india-vs-new-zealand-5th-t20i-highlights-ishan-kishan-ton-arshdeep-singh-fifer-seal-4-1-series-win-for-india-7649774)
2.  **Pakistan Dominates Australia by 90 Runs to Clinch T20 Series**
    [https://www.beaumontenterprise.com/sports/article/pakistan-routs-australia-by-90-runs-to-clinch-t20-18640161.php](https://www.beaumontenterprise.com/sports/article/pakistan-routs-australia-by-90-runs-to-clinch-t20-18640161.php)

### Business
1.  **Gold Prices See Sharp Drop on MCX After Reaching Record Highs**
    [https://www.timesnownews.com/business/gold-price-today-january-31-2026-sharp-fall-after-record-highs-check-24k-22k-18k-prices-in-delhi-mumbai-chennai-bangalore-hyderabad-and-other-major-cities-article-107310034](https://www.timesnownews.com/business/gold-price-today-january-31-2026-sharp-fall-after-record-highs-check-24k-22k-18k-prices-in-delhi-mumbai-chennai-bangalore-hyderabad-and-other-major-cities-article-107310034)
2.  **NSE IPO Back on Track as SEBI Gives Green Signal After 10 Years**
    [https://www.thesundayguardian.com/business/school-assembly-news-headlines-today-january-31-top-national-business-news-sports-news-world-news-with-weather-updates-thought-of-the-day](https://www.thesundayguardian.com/business/school-assembly-news-headlines-today-january-31-top-national-business-news-sports-news-world-news-with-weather-updates-thought-of-the-day)

### Technology
1.  **US Authorities Reportedly Investigate Claims Meta Can Read Encrypted WhatsApp Messages**
    [https://www.theguardian.com/technology/2026/jan/31/us-authorities-reportedly-investigate-claims-that-meta-can-read-encrypted-whatsapp-messages](https://www.theguardian.com/technology/2026/jan/31/us-authorities-reportedly-investigate-claims-that-meta-can-read-encrypted-whatsapp-messages)
2.  **UK and Japan Launch New Strategic Cyber Partnership to Boost Security and Protect Economies**
    [https://www.gov.uk/government/speeches/pm-press-statement-alongside-japanese-prime-minister-sanae-takaichi-31-january-2026](https://www.gov.uk/government/speeches/pm-press-statement-alongside-japanese-prime-minister-sanae-takaichi-31-january-2026)

### Local (Mumbai)
1.  **Sunetra Pawar Sworn in as Maharashtra's First Woman Deputy Chief Minister**
    [https://www.thehindu.com/news/national/top-news-of-the-day-sunetra-pawar-sworn-in-as-first-woman-deputy-cm-of-maharashtra-rybakina-becomes-first-kazakh-to-win-australian-open-womens-singles-title-and-more/article69234812.ece](https://www.thehindu.com/news/national/top-news-of-the-day-sunetra-pawar-sworn-in-as-first-woman-deputy-cm-of-maharashtra-rybakina-becomes-first-kazakh-to-win-australian-open-womens-singles-title-and-more/article69234812.ece)
2.  **Woman Dies After Falling Off Crowded Local Train in Thane District**
    [https://www.thehindu.com/news/national/woman-dies-after-falling-off-crowded-local-train-in-maharashtras-thane-district/article69234676.ece](https://www.thehindu.com/news/national/woman-dies-after-falling-off-crowded-local-train-in-maharashtras-thane-district/article69234676.ece)

--------------------------------------------------



In [23]:
### NOT RUNNING THIS ONE EITHER. SEE SEQUENTIAL AGENT BELOW ###
## TESTING ONLY
async def run_root_agent():
    # Create a new, single-use session for this query
    root_session = await session_service.create_session(
        app_name=root_agent.name,
        user_id=my_user_id
    )

    # Note the new budget constraint in the query!
    query = "Find me top news headlines and weather updates for Mumbai for the day"
    print(f"🗣️ User Query: '{query}'")

    await run_agent_query(root_agent, query, root_session, my_user_id)

await run_root_agent()
## RUNNING TILL HERE IS FINE. Lets move this to adk web to trace, before cleaning up.

🗣️ User Query: 'Find me top news headlines and weather updates for Mumbai for the day'

🚀 Running query for agent: 'NewsWeatherReportPipeline' in session: 'a9ef8d74-9dc5-4a16-a4f1-6aa3a5056d15'...

--------------------------------------------------
✅ Final Response:


## Summary of Recent Sustainable Technology Advancements

### News Results
(Based on news_agent's findings)
The news_agent reported various developments across several sectors. In World News, the Philippines and China have resumed talks amidst existing tensions, and a UN Watchdog has warned that the Ukraine War remains the biggest threat to nuclear safety. For Sports (Cricket), India secured a 46-run victory against New Zealand in the 5th T20I, with notable performances from Ishan Kishan and Arshdeep. Australia also made two changes to its T20 World Cup squad, including replacing Pat Cummins. Business news indicated that business growth and innovation are key to boosting India's productivity, and Public Sector Banks have outperformed private peers in the Q3 profit race. In Technology, there is discussion on how workers can compete with machines in the AI era and how LEO Connectivity is reshaping airline strategy and passenger expectations. Locally in Mumbai, a cab driver was arrested for allegedly overcharging a US tourist, and Mumbai Local Trains are set to receive 100 new rakes to ease crowding. The news_agent also included a weather update for Mumbai for *Saturday, January 31, 2026*, which reported dense smog, unseasonably cold conditions, and a 'severe' Air Quality Index (AQI) of 320, advising residents to limit outdoor exposure due to pollution concerns amidst cloudy and hazy conditions with temperatures between 21°C and 31°C.

### Weather Results
(Based on weather_agent's findings)
The weather_agent provided a forecast for Mumbai for *Sunday, February 1, 2026*, indicating a beautiful day with mostly sunny skies throughout, transitioning to clear conditions at night. Rain is expected to be a very rare occurrence. Temperatures are predicted to be pleasant, starting around 23°C to 26°C in the morning, rising to 28°C to 30°C in the afternoon, and gently cooling to 24°C to 26°C in the evening. The Air Quality Index (AQI) in Mumbai is currently in the 'Unhealthy' category, with real-time readings around 179 to 189, suggesting health effects for sensitive groups and potential difficulty breathing for healthy individuals with prolonged exposure. However, some forecasts suggest that air quality is likely to be in the 'Moderate' category from January 31st to February 2nd.

### Overall Summary
The reports highlight a mix of global geopolitical shifts, national sports and economic trends, and local Mumbai developments including an upcoming boost to local trains. Mumbai's weather forecast for February 1, 2026, indicates a sunny and pleasant day, while air quality remains a concern, following a 'severe' AQI reported for January 31, 2026, and an 'unhealthy' current status with potential for improvement.

--------------------------------------------------



In [22]:
# --- 4. Create the SequentialAgent (Orchestrates the overall flow) ---
# This is the main agent that will be run. It first executes the ParallelAgent
# to populate the state, and then executes the MergerAgent to produce the final output.

# The ValidationError indicates that the `root_agent` (which was a ParallelAgent
# containing weather and news agents) already has a parent agent named 'NewsAndWeatherPipeline'.
# An agent instance can only be a sub-agent of one parent at a time.
# To fix this, we need to create a new instance of the ParallelAgent to be used
# as a sub-agent within this new SequentialAgent.

# Create a new ParallelAgent instance for weather and news
def create_parallel_weather_news_agent():
  return ParallelAgent(
      name="ParallelWeatherNews", # Give it a distinct name for clarity
      description="Runs weather and news agents in parallel to gather information.",
      sub_agents=[weather_agent, news_agent]
  )

parallel_weather_news_agent = create_parallel_weather_news_agent()
print(f"created {parallel_weather_news_agent.name} agent to run news, weather in parallel for given city")

def create_sequential_root_agent():
  return SequentialAgent(
      name="NewsWeatherReportPipeline", # This will be the main orchestrating agent
      # Run the newly created parallel agent first, then the merger agent
      sub_agents=[parallel_weather_news_agent, merger_agent], # Use the newly created parallel agent
      description="Orchestrates the fetching of news and weather information in parallel, then merges the results into a single report."
  )

# Assign the newly created SequentialAgent to the `root_agent` variable.
# This variable now represents the top-level orchestrator of the entire pipeline.
root_agent = create_sequential_root_agent()
print(f"🧞 Agent '{root_agent.name}' is created and ready to generate news, weather and summarise/ merge into single report!")

ValidationError: 1 validation error for ParallelAgent
  Value error, Agent `weather_agent` already has a parent agent, current parent: `ParallelWeatherNews`, trying to add: `ParallelWeatherNews` [type=value_error, input_value={'name': 'ParallelWeather...l_error_callback=None)]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error

In [24]:
# Running root/ sequential agent
## TESTING
async def run_root_agent():
    # Create a new, single-use session for this query
    root_session = await session_service.create_session(
        app_name=root_agent.name,
        user_id=my_user_id
    )
    print("root session")
    print(root_session)
    print("looks good? \n")

    # Note the new budget constraint in the query!
    query = "Find me top news headlines and weather updates for the day"
    print(f"🗣️ User Query: '{query}'")

    await run_agent_query(news_agent, query, root_session, my_user_id)

await run_root_agent()

root session
id='47648a70-d9dd-4835-9d88-f55edb96e478' app_name='NewsAndWeatherPipeline' user_id='rodi_adventurer_001' state={} events=[] last_update_time=1769781413.638791
looks good? 

🗣️ User Query: 'Find me top news headlines and weather updates for the day'

🚀 Running query for agent: 'day_trip_agent' in session: '47648a70-d9dd-4835-9d88-f55edb96e478'...

--------------------------------------------------
✅ Final Response:


An error occurred: Session not found: 47648a70-d9dd-4835-9d88-f55edb96e478

--------------------------------------------------

